In [1]:
import numpy as np
head_embed=np.load('head_biencodings.npy').item()
body_embed=np.load('body_biencodings.npy').item()

In [2]:

from baseline.utils import dataset, generate_test_splits, score
train_dataset = dataset.DataSet()
test_dataset = dataset.DataSet('test')

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [3]:
len(train_dataset.stances)

49972

In [5]:
#Creating features {(unews.vhead) concat (|unews-vhead|) concat (TF-IDF) concat (Handmade)}

features_train = np.memmap('feattrain.myarray', dtype=float, mode='w+',shape=(49972,19645))
labels_train=[]
ftrain=[]
cnt=0
for s in train_dataset.stances:
    i=s['Body ID']
    a=head_embed[s['Headline']]
    b=np.average(body_embed[i],axis=0)
    b=np.nan_to_num(b)
    x1=np.multiply(a,b)
    x2=np.absolute(np.subtract(a,b))
    x3=tfidf_train[cnt]
    x4=handmade_train[cnt]
    ftrain.append(np.concatenate([x1,x2,x3,x4]))
    labels_train.append(s['Stance'])
    cnt+=1

features_train[:]=ftrain[:]
    

ValueError: could not broadcast input array from shape (9644) into shape (19645)

In [7]:

cnt=0
features_test = np.memmap('feattest.myarray', dtype=float, mode='w+',shape=(25413,19645))
ftest=[]
labels_test=[]
for s in test_dataset.stances:
    i=s['Body ID']
    a=head_embed[s['Headline']]
    b=np.average(body_embed[i],axis=0)
    b=np.nan_to_num(b)
    x1=np.multiply(a,b)
    x2=np.absolute(np.subtract(a,b))
    x3=tfidf_test[cnt]
    x4=handmade_test[cnt]
    ftest.append(np.concatenate([x1,x2,x3,x4]))
    labels_test.append(s['Stance'])
    cnt+=1
    
features_test[:]=ftest[:]

del features_test
del features_train

In [4]:
path='/home/kamal/CS249/Codes/'
tfidf_train=np.load(path+'train_tf_idf_full.npy')
handmade_train=np.load(path+'train_hand_baseline.npy')
tfidf_test=np.load(path+'test_tf_idf_full.npy')
handmade_test=np.load(path+'test_hand_baseline.npy')

In [47]:
import GraphViz
import pydot_ng as pydot
pydot.find_graphviz()

ModuleNotFoundError: No module named 'GraphViz'

In [13]:
from keras.layers import Input, Dense,Dropout
from keras.models import Model,Sequential 

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [31]:
model = Sequential()
model.add(Dense(100, input_dim=19645, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encodertrain = LabelEncoder()
    
encodertrain.fit(labels_train)
encoded_Y = encodertrain.transform(labels_train)
Y_train_labels = np_utils.to_categorical(encoded_Y)
  

encodertest = LabelEncoder()
encodertest.fit(labels_test)
encoded_Y = encodertest.transform(labels_test)
Y_test_labels = np_utils.to_categorical(encoded_Y)

In [8]:
featstrain=np.memmap('feattrain.myarray', dtype=float, mode='r',shape=(49972,19645))

In [18]:
featstest=np.memmap('feattest.myarray', dtype=float, mode='r',shape=(25413,19645))

In [20]:
print(len(featstrain))
print(len(featstest))
print(len(featstrain[0]))
print(len(featstest[0]))


49972
25413
19645
19645


In [16]:
#print(len(features_train[0]))
print(len(Y_train_labels))
#print(len(features_test))
print(len(Y_test_labels))

49972
25413


In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               1964600   
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 204       
Total params: 1,969,854
Trainable params: 1,969,854
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(featstrain,np.array(Y_train_labels),batch_size=128,epochs=10,verbose=1)

Epoch 1/10
  768/49972 [..............................] - ETA: 31:57 - loss: 0.9848 - acc: 0.7161

KeyboardInterrupt: 

In [23]:
def evaluate_answer(model, feats, truelabels,lenc):
    predicted = model.predict(np.array(feats))
    predicted = np.argmax(predicted,axis =-1)
    ground = list()
    pred = list()
    t=np.argmax(truelabels,axis =-1)
    for i in predicted:
        pred.append(encodertest.inverse_transform(i))
    for i in t:
        ground.append(encodertest.inverse_transform(i))
    score.report_score(ground, pred)

In [24]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

evaluate_answer(model,featstest, Y_test_labels,encodertest)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    724    |     8     |    966    |    205    |
-------------------------------------------------------------
| disagree  |    169    |    12     |    326    |    190    |
-------------------------------------------------------------
|  discuss  |    280    |     4     |   3769    |    411    |
-------------------------------------------------------------
| unrelated |    17     |     0     |    268    |   18064   |
-------------------------------------------------------------
Score: 9459.25 out of 11651.25	(81.18656796481065%)


In [29]:
encodertest.classes_

array(['agree', 'disagree', 'discuss', 'unrelated'], dtype='<U9')